<a href="https://colab.research.google.com/github/ArshiaSali/Approximate-Nearest-Neighbor-Search/blob/main/ANN_Search_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Approximate Nearest Neighbor Search Algorithms**

### **Dataset Used**

The dataset contain users answering questions: An interaction is defined as a user answering a given question.

The following datasets from the StackExchange network are available:

 - **CrossValidated:** From stats.stackexchange.com. Approximately 9000 users, 72000 questions, and 70000 answers.

 - **StackOverflow:** From stackoverflow.stackexchange.com. Approximately 1.3M users, 11M questions, and 18M answers.


Here CrossValidated Dataset is considered for analysis.


### **Import packages and Libraries**

In [1]:
import pickle
!pip install lightfm
from lightfm import LightFM
from lightfm.datasets import fetch_stackexchange

!pip install nmslib
import nmslib

!pip install faiss-cpu --no-cache
import faiss

!pip install annoy
import annoy


     |████████████████████████████████| 310 kB 5.2 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705355 sha256=f5852850d96d97ada19ba3486a7a0e646f6ff001cfca620d4770ba4377abaf12
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm
     |████████████████████████████████| 13.5 MB 1.4 MB/s 
     |████████████████████████████████| 188 kB 51.3 MB/s 
     |████████████████████████████████| 8.4 MB 4.5 MB/s 
     |████████████████████████████████| 646 kB 5.3 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391687 sha256=668ae0d578869fa71a3b069f55c57b7e42782bae22acbec443ad340a6d98c2bc
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [2]:
!pip install stackapi


  Created wheel for stackapi: filename=StackAPI-0.2.0-py3-none-any.whl size=5856 sha256=7f84a460f4ab99841796927032ec26f9f31060347d101a9e9bbee307db9f8b7c
  Stored in directory: /root/.cache/pip/wheels/ec/db/60/df42a65853e3581c26a2fbb2012a228cb8e267369a3b9ca44d
Successfully built stackapi


### **Import Dataset**

In [3]:
stackexchange = fetch_stackexchange('crossvalidated')

In [4]:
train = stackexchange['train']
test = stackexchange['test']

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=stackexchange['item_features'], epochs=20 )

item_vectors = stackexchange['item_features'] * model.item_embeddings

from termcolor import cprint
print_red_on_cyan = lambda x: cprint(x, 'yellow', 'on_grey')

In [5]:
stackexchange['item_feature_labels']

array(['question_id:0', 'question_id:1', 'question_id:2', ...,
       'question_id:72357', 'question_id:72358', 'question_id:72359'],
      dtype='<U17')

### **Pickle the Model**

In [6]:
with open('stack_exchange.pickle', 'wb') as f:
    pickle.dump({"name": stackexchange['item_feature_labels'], "vector": item_vectors}, f)

### **Function to Load Pickled Model**

In [7]:
def load_data():
    with open('stack_exchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['question_id:0', 'question_id:1', 'question_id:2', ...,
        'question_id:72357', 'question_id:72358', 'question_id:72359'],
       dtype='<U17'),
 'vector': array([[ 0.05906702,  0.02375606,  0.07332865, ..., -0.01101989,
         -0.0683156 ,  0.08668615],
        [ 0.13899006,  0.11184364,  0.05210828, ..., -0.10418914,
         -0.16255663,  0.00553737],
        [-0.01872136, -0.13028923,  0.11398011, ...,  0.05952855,
         -0.16864328, -0.07374636],
        ...,
        [-0.0119923 ,  0.02600207, -0.02288014, ..., -0.00684505,
          0.02163115, -0.02398989],
        [ 0.00402075,  0.00426441,  0.00309685, ..., -0.01254157,
          0.00961653, -0.00694339],
        [-0.0084562 , -0.00758704,  0.01070431, ...,  0.00627414,
         -0.01246643,  0.01460843]], dtype=float32)}

##**Locality Sensitive Hashing**

**LSH** refers to a family of functions (known as LSH families) to hash data points into buckets so that data points near each other are located in the same buckets with high probability, while data points far from each other are likely to be in different buckets. 

This makes it easier to identify observations with various degrees of similarity.

LSH has many applications, including:


**Near-duplicate detection:**

 LSH is commonly used to deduplicate large quantities of documents, webpages, and other files.

**Genome-wide association study:**

 Biologists often use LSH to identify similar gene expressions in genome databases.

**Large-scale image search:** 

Google used LSH along with PageRank to build their image search technology VisualRank.

**Audio/video fingerprinting:** 

In multimedia technologies, LSH is widely used as a fingerprinting technique A/V data.


###**Index Class**

In [8]:
#LSH
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

###**Build Index**

In [9]:
index = LSHIndex(data["vector"], data["name"])
index.build()

###**Query for Similar Questions**

Here, we are querying the index to return questions that are similar to the Question Id - 90.

In [10]:
question_vector, question_id = data['vector'][90:91], data['name'][90]
simlar_question_ids = '\n* '.join(index.query(question_vector))
print(f"The most similar questions to {question_id} are:\n* {simlar_question_ids}")

The most similar questions to question_id:90 are:
* question_id:155
* question_id:189
* question_id:172
* question_id:51
* question_id:6
* question_id:240
* question_id:253
* question_id:323
* question_id:90
* question_id:22


### **Visualize Similar Questions**

In [11]:
from stackapi import StackAPI
QUESTION_IDS = [
514
,90
,594] 
SITENAME = 'stackoverflow'
SITE = StackAPI(SITENAME)

for qn in QUESTION_IDS:
  question = SITE.fetch('questions/{ids}', ids=[qn], filter='withbody')
  print_red_on_cyan("Question : " + question['items'][0]['title'])
  print()

Question : Frequent SystemExit in Ruby when making HTTP calls

Question : How do you branch and merge with TortoiseSVN?

Question : cx_Oracle: How do I iterate over a result set?



### **Inference** 
We can see the questions similar to Question Id 9 - **How do you branch and merge with TortoiseSVN?** using Locality Sensitive Hashing Technique. 

LSH refers to functions to hash data points into buckets so that data points near each other are located in the same buckets with high probability, while data points far from each other are likely to be in different buckets.


##**Exhaustive Search**

**Exhaustive search**- Comparing each point to every other point, which will require Linear query time (the size of the dataset).

The only available method for guaranteed retrieval of the exact nearest neighbor is exhaustive search (due to the curse of dimensionality.)

###**Index Class**

In [12]:
#Exhaustive Search
class ExactIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

###**Build Index**

In [13]:
index = ExactIndex(data["vector"], data["name"])
index.build()

###**Query for Similar Questions**

Here, we are querying the index to return questions that are similar to the Question Id - 90.

In [14]:
question_vector, question_id = data['vector'][90:91], data['name'][90]
simlar_question_ids = '\n* '.join(index.query(question_vector))
print(f"The most similar questions to {question_id} are:\n* {simlar_question_ids}")

The most similar questions to question_id:90 are:
* question_id:90
* question_id:296
* question_id:3129
* question_id:916
* question_id:3356
* question_id:6
* question_id:569
* question_id:433
* question_id:3050
* question_id:444


### **Visualize Similar Questions**

In [15]:
from stackapi import StackAPI
QUESTION_IDS = [90,5328,17681,6440] # For example
SITENAME = 'stackoverflow'
SITE = StackAPI(SITENAME)

for qn in QUESTION_IDS:
  question = SITE.fetch('questions/{ids}', ids=[qn], filter='withbody')
  print_red_on_cyan("Question : " + question['items'][0]['title'])
  print()


Question : How do you branch and merge with TortoiseSVN?

Question : Why can&#39;t I use a try block around my super() call?

Question : WebSVN with VisualSVN Server, anyone gotten authentication to work?

Question : .NET 3.5 Redistributable -- 200 MB? Other options?



### **Inference** 
We can see the questions similar to Question Id 9 - **How do you branch and merge with TortoiseSVN?** using Exhaustive Search Technique. 

Exhaustive search- Comparing each point to every other point, which will require Linear query time (the size of the dataset).


##**Product Quantization**

**Product quantization** is an effective vector quantization
approach to compactly encode high-dimensional vectors
for fast approximate nearest neighbor (ANN) search. 

The
essence of product quantization is to decompose the original high-dimensional space into the Cartesian product of
a finite number of low-dimensional subspaces that are then
quantized separately.

Optimal space decomposition is important for the performance of ANN search, but still remains unaddressed. In this paper, we optimize product quantization by minimizing quantization distortions w.r.t. the space decomposition and the quantization codebooks.

###**Index Class**

Create the index class where we can control the subvector_size, number_of_partitions and search_in_x_partitions

In [16]:
#Product Quantization
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

###**Build Index**

In [17]:
index = IVPQIndex(data["vector"], data["name"])
index.build()

### **Query for Similar Questions**

Here, we are querying the index to return questions that are similar to the Question Id - 90.

In [18]:
question_vector, question_id = data['vector'][90:91], data['name'][90]
simlar_question_ids = '\n* '.join(index.query(question_vector))
print(f"The most similar questions to {question_id} are:\n* {simlar_question_ids}")

The most similar questions to question_id:90 are:
* question_id:90
* question_id:3678
* question_id:5508
* question_id:1582
* question_id:6280
* question_id:569
* question_id:306
* question_id:1886
* question_id:2300
* question_id:6440


### **Visualize Similar Questions**

In [19]:
from stackapi import StackAPI
QUESTION_IDS = [9
,6
,90
,5328
,9136
] 
SITENAME = 'stackoverflow'
SITE = StackAPI(SITENAME)

for qn in QUESTION_IDS:
  question = SITE.fetch('questions/{ids}', ids=[qn], filter='withbody')
  print_red_on_cyan("Question : " + question['items'][0]['title'])
  print()


Question : How do I calculate someone&#39;s age based on a DateTime type birthday?

Question : Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?

Question : How do you branch and merge with TortoiseSVN?

Question : Why can&#39;t I use a try block around my super() call?

Question : Enterprise Library CacheFactory.GetCacheManager Throws Null Ref



### **Inference** 
We can see the questions similar to Question Id 9 - **How do you branch and merge with TortoiseSVN?** using Product Quantization Technique. 

Product quantization is an effective vector quantization approach to compactly encode high-dimensional vectors for fast approximate nearest neighbor (ANN) search.

##**Trees and Graphs**

**Tree-based algorithms** are one of the most common strategies when it comes to ANN. They construct forests (collection of trees) as their data structure by **splitting the dataset into subsets**.


One of the most prominent solutions out there is **Annoy**, which uses trees (more accurately forests) to enable Spotify’ music recommendations. 

In Annoy, in order to construct the index we create a forest (aka many trees) Each tree is constructed in the following way, we pick two points at random and split the space into two by their hyperplane, we keep splitting into the subspaces recursively until the points associated with a node is small enough.


In order to search the constructed index, the forest is traversed in order to obtain a set of candidate points from which the closest to the query point is returned.



###**Index class**

**Annoy Usage**


We use annoy library. Most of the logic is in the build method (index creation), where the accuracy-performance tradeoff is controlled by:

**number_of_trees** — the number of binary trees , a larger value will give more accurate results, but larger indexes.

**search_k** — the number of binary trees we search for each point, a larger value will give more accurate results, but will take a longer time to return.

In [20]:
#Trees
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

###**Build Index**

In [21]:
index = AnnoyIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  # Remove the CWD from sys.path while we load stuff.


###**Query for Similar Questions**

Here, we are querying the index to return questions that are similar to the Question Id - 90.

In [22]:
question_vector, question_id = data['vector'][90], data['name'][90]
simlar_question_ids = '\n* '.join(index.query(question_vector))
print(f"The most similar questions to {question_id} are:\n* {simlar_question_ids}")

The most similar questions to question_id:90 are:
* question_id:90
* question_id:3129
* question_id:102
* question_id:450
* question_id:1947
* question_id:3451
* question_id:3789
* question_id:3366
* question_id:253
* question_id:49022


### **Visualize Similar Questions**

In [23]:
from stackapi import StackAPI
QUESTION_IDS = [90
,5328
,13
,34] 
SITENAME = 'stackoverflow'
SITE = StackAPI(SITENAME)

for qn in QUESTION_IDS:
  question = SITE.fetch('questions/{ids}', ids=[qn], filter='withbody')
  print_red_on_cyan("Question : " + question['items'][0]['title'])
  print()


Question : How do you branch and merge with TortoiseSVN?

Question : Why can&#39;t I use a try block around my super() call?

Question : Determine a user&#39;s timezone

Question : How to unload a ByteArray using Actionscript 3?



### **Inference** 
We can see the questions similar to Question Id 9 - **How do you branch and merge with TortoiseSVN?** using Trees and Graphs Technique. 

Tree-based algorithms are one of the most common strategies when it comes to ANN. They construct forests (collection of trees) as their data structure by splitting the dataset into subsets.

##**HNSW**

An **HNSW index** consists of navigable small world graphs in a hierarchy. Each document in the index is represented by a single graph node. Each node has an array of levels, from level 0 to n. The number of levels is constant during the lifetime of the node and is drawn randomly when the node is created. All nodes have at least level 0. At each level there is a link array which contains the document ids of the nodes it is connected to at that level.

###**Index class**

In [24]:
#hsnw
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

###**Build Index**

In [25]:
index = NMSLIBIndex(data["vector"], data["name"])
index.build()

###**Query for Similar Questions**

Here, we are querying the index to return questions that are similar to the Question Id - 90.

In [26]:
question_vector, question_id = data['vector'][90], data['name'][90]
simlar_question_ids = '\n* '.join(index.query(question_vector))
print(f"The most similar questions to {question_id} are:\n* {simlar_question_ids}")

The most similar questions to question_id:90 are:
* question_id:90
* question_id:296
* question_id:14
* question_id:433
* question_id:3129
* question_id:1394
* question_id:22
* question_id:102
* question_id:762
* question_id:6


### **Visualize Similar Questions**

In [27]:
from stackapi import StackAPI
QUESTION_IDS = [
                90,6440,5328,17681
] 
SITENAME = 'stackoverflow'
SITE = StackAPI(SITENAME)

for qn in QUESTION_IDS:
  question = SITE.fetch('questions/{ids}', ids=[qn], filter='withbody')
  print_red_on_cyan("Question : " + question['items'][0]['title'])
  print()


Question : How do you branch and merge with TortoiseSVN?

Question : .NET 3.5 Redistributable -- 200 MB? Other options?

Question : Why can&#39;t I use a try block around my super() call?

Question : WebSVN with VisualSVN Server, anyone gotten authentication to work?



### **Inference** 
We can see the questions similar to Question Id 9 - **How do you branch and merge with TortoiseSVN?** using HNSW Technique. 

An HNSW index consists of navigable small world graphs in a hierarchy. Each document in the index is represented by a single graph node. Each node has an array of levels, from level 0 to n. The number of levels is constant during the lifetime of the node and is drawn randomly when the node is created. All nodes have at least level 0. At each level there is a link array which contains the document ids of the nodes it is connected to at that level.